In [ ]:
%matplotlib inline
import matpotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

class Problem(object):
    def __init__(self, initial=None, gola=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)
        
    def actions(self, state): raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state): return state == self.goal
    def action_cost(self, s,a,s1): return 1
    def h(self, node): return 0

class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)
        
    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost

failure = Node('failure', path_cost=math.inf) # indicates an algorithm couldn't find a solution
cutoff  = Node('cutoff', path_cost=math.inf) # indicates iterative deepening search was cut off

def expand(problem, node):
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        
        
def path_actions(node):
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]

def path_states(node):
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

FIFOQueue = deque
LIFOQueue = list

class PriorityQueue:
    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
            
    def add(self, item):
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)
        
    def pop(self):
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)


def breadth_first_search(problem):
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure